In [9]:
import tensorflow as tf

In [10]:
mnist = tf.keras.datasets.mnist

In [24]:
import tensorflow_datasets as tfds

In [27]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /home/john/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


2021-08-23 12:05:47.278978: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-23 12:05:47.279221: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-23 12:05:47.279267: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-OHKTTDHA): /proc/driver/nvidia/version does not exist
2021-08-23 12:05:47.280408: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [28]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [30]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [31]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6


2021-08-23 12:07:19.165581: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


469/469 [==============================] - 2s 2ms/step - loss: 0.3539 - sparse_categorical_accuracy: 0.9030 - val_loss: 0.1878 - val_sparse_categorical_accuracy: 0.9435
Epoch 2/6
469/469 [==============================] - 1s 1ms/step - loss: 0.1602 - sparse_categorical_accuracy: 0.9539 - val_loss: 0.1356 - val_sparse_categorical_accuracy: 0.9600
Epoch 3/6
469/469 [==============================] - 1s 1ms/step - loss: 0.1145 - sparse_categorical_accuracy: 0.9668 - val_loss: 0.1123 - val_sparse_categorical_accuracy: 0.9644
Epoch 4/6
469/469 [==============================] - 1s 1ms/step - loss: 0.0898 - sparse_categorical_accuracy: 0.9738 - val_loss: 0.0926 - val_sparse_categorical_accuracy: 0.9722
Epoch 5/6
469/469 [==============================] - 1s 1ms/step - loss: 0.0717 - sparse_categorical_accuracy: 0.9790 - val_loss: 0.0875 - val_sparse_categorical_accuracy: 0.9733
Epoch 6/6
469/469 [==============================] - 1s 1ms/step - loss: 0.0589 - sparse_categorical_accuracy: 0.98

In [33]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0773 - sparse_categorical_accuracy: 0.9763


[0.07729553431272507, 0.9763000011444092]